In [1]:
import tensorflow as tf

In [6]:
from tensorflow import keras
from tensorflow.keras import datasets
import os

In [7]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [9]:
# 加载数据集
# x: [60k, 28, 28]
# y: [60k]
(x, y), _ = datasets.mnist.load_data()
# x: [0~255] => [0~1.]
x = tf.convert_to_tensor(x, dtype=tf.float32) / 255.
y = tf.convert_to_tensor(y, dtype=tf.int32)

x.shape, y.shape, tf.reduce_min(x), tf.reduce_max(x), tf.reduce_min(y), tf.reduce_max(y)

(TensorShape([60000, 28, 28]),
 TensorShape([60000]),
 <tf.Tensor: id=14, shape=(), dtype=float32, numpy=0.0>,
 <tf.Tensor: id=16, shape=(), dtype=float32, numpy=1.0>,
 <tf.Tensor: id=18, shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: id=20, shape=(), dtype=int32, numpy=9>)

In [17]:
train_db = tf.data.Dataset.from_tensor_slices((x, y)).batch(128)
train_iter = iter(train_db)
sample = next(train_iter)
sample[0].shape, sample[1].shape

(TensorShape([128, 28, 28]), TensorShape([128]))

In [20]:
# [b, 784] => [b, 256] => [b, 128] => [b, 10]
# [dim_in, dim_out], [dim_out]
w1 = tf.Variable(tf.random.truncated_normal([784, 256], stddev=0.1))
b1 = tf.Variable(tf.zeros([256]))
w2 = tf.Variable(tf.random.truncated_normal([256, 128], stddev=0.1))
b2 = tf.Variable(tf.zeros([128]))
w3 = tf.Variable(tf.random.truncated_normal([128, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

In [21]:
lr = 1e-3

for epoch in range(10):
    for step, (x, y) in enumerate(train_db):
        # x: [128, 28, 28]
        # y: [128]
        
        # [b, 28, 28] => [b, 28*28]
        x = tf.reshape(x, [-1, 28*28])
        
        with tf.GradientTape() as tape: # tf.Variable
            # x: [b, 28*28]
            # h1 = x@w1 + b1
            # [b, 784]@[784, 256] + [256] => [b, 256] + [256] => [b, 256] + [b, 256]
            h1 = x@w1 + tf.broadcast_to(b1, [x.shape[0], 256])
            h1 = tf.nn.relu(h1)
            # [b, 256] => [b, 128]
            h2 = h1@w2 + b2
            h2 = tf.nn.relu(h2)
            # [b, 128] => [b, 10]
            out = h2@w3 + b3
            
            # compute loss
            # out: [b, 10]
            # y: [b] => [b, 10]
            y_onehot = tf.one_hot(y, depth=10)
            
            # mse = mean(sum(y-out)^2)
            # [b, 10]
            loss = tf.square(y_onehot - out)
            # mean: scalar
            loss = tf.reduce_mean(loss)
        
        # compute gradients
        grads = tape.gradient(loss, [w1, b1, w2, b2, w3, b3])
        # print(grads)
        # w1 = w1 - lr * w1_grads 
        w1.assign_sub(lr * grads[0])
        b1.assign_sub(lr * grads[1])
        w2.assign_sub(lr * grads[2])
        b2.assign_sub(lr * grads[3])
        w3.assign_sub(lr * grads[4])
        b3.assign_sub(lr * grads[5])
        
        if step % 100 == 0:
            print(epoch, step, 'loss:', float(loss))

0 0 loss: 0.38031715154647827
0 100 loss: 0.20235104858875275
0 200 loss: 0.17986197769641876
0 300 loss: 0.16451908648014069
0 400 loss: 0.16647419333457947
1 0 loss: 0.14627186954021454
1 100 loss: 0.14956852793693542
1 200 loss: 0.14416329562664032
1 300 loss: 0.13515308499336243
1 400 loss: 0.14004984498023987
2 0 loss: 0.12516537308692932
2 100 loss: 0.1299421787261963
2 200 loss: 0.1252199113368988
2 300 loss: 0.11830594390630722
2 400 loss: 0.12403054535388947
3 0 loss: 0.1119314655661583
3 100 loss: 0.11756940186023712
3 200 loss: 0.11326563358306885
3 300 loss: 0.10732883214950562
3 400 loss: 0.11334268748760223
4 0 loss: 0.10278837382793427
4 100 loss: 0.10878705978393555
4 200 loss: 0.10486634075641632
4 300 loss: 0.09952405840158463
4 400 loss: 0.10564304888248444
5 0 loss: 0.09605226665735245
5 100 loss: 0.10214515775442123
5 200 loss: 0.09855443239212036
5 300 loss: 0.09365519136190414
5 400 loss: 0.09979870170354843
6 0 loss: 0.09082411229610443
6 100 loss: 0.09686379134